## USE GPU

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.4 MB/s eta 0:00:00


In [2]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#Loading Mutilingual BERT





In [3]:
# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained('bert-base-multilingual-uncased')

In [4]:
def get_bert_embeddings(text):
    # Ensure text is a string
    text = str(text)
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1)

# Assume text1 and text2 are your text data in Spanish or English
text1 = "This is a sample text in English."
text2 = "Este es un texto de ejemplo en español."

# Get BERT embeddings
embedding1 = get_bert_embeddings(text1)
embedding2 = get_bert_embeddings(text2)


In [5]:
# Convert torch tensors to numpy arrays for use with sklearn
embedding1_np = embedding1.numpy()
embedding2_np = embedding2.numpy()

# Compute cosine similarity
similarity = cosine_similarity(embedding1_np, embedding2_np)
print(f'Cosine Similarity: {similarity[0][0]}')
print(similarity)

Cosine Similarity: 0.7640103101730347
[[0.7640103]]


Let's Start Building

##Getting the Data

In [6]:
import zipfile
!wget https://www.dropbox.com/s/yle22nlwxhgf7pb/color.zip?dl=0&file_subpath=%2Fcolor%2Fexperiment1-dataset-color-of-concrete-objects.txt
# Specify the name of the ZIP file
zip_file = './color.zip?dl=0'

# Specify the directory where you want to extract the contents
extract_path = './'

# Create the directory if it doesn't exist
import os
os.makedirs(extract_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f'Contents of {zip_file} have been extracted to {extract_path}')
%cd color



--2023-11-08 12:03:46--  https://www.dropbox.com/s/yle22nlwxhgf7pb/color.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/yle22nlwxhgf7pb/color.zip [following]
--2023-11-08 12:03:46--  https://www.dropbox.com/s/raw/yle22nlwxhgf7pb/color.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc09151b4e830158fac1b0cb8094.dl.dropboxusercontent.com/cd/0/inline/CHLOjmKFG3CSXmBXXjoEGnTwhoJbB2wYXP5hM_K-w3tB_-VWYqFX_MoO1fHKdFRpoyoJFiNL_cvrbvVjX60sX6lHFfX3Uef1--SFelsDJzY9hqsSER4P4C3b5riKkC1VzBA/file# [following]
--2023-11-08 12:03:47--  https://uc09151b4e830158fac1b0cb8094.dl.dropboxusercontent.com/cd/0/inline/CHLOjmKFG3CSXmBXXjoEGnTwhoJbB2wYXP5hM_K-w3tB_-VWYqFX_MoO1fHKdFRpoyoJFiNL_cvrbvVjX60sX6lHFfX3Uef1--SFelsDJzY9hqsSER4P4C3b5riKkC1

In [7]:
import pandas as pd
df = pd.read_csv('experiment1-dataset-color-of-concrete-objects.txt',sep='\t', names =['noun','color'])
bless = df

In [8]:
# Add true_similarity column
def compute_cosine_similarity(noun, color):
    noun_vector = get_bert_embeddings(noun)
    color_vector = get_bert_embeddings(color)
    noun_vector_np = noun_vector.numpy()
    color_vector_np = color_vector.numpy()
    similarity = cosine_similarity(noun_vector_np, color_vector_np)
    return similarity[0][0]

bless['true_similarity'] = bless.apply(lambda row: compute_cosine_similarity(row['noun'], row['color']), axis=1)

color_list = ['black','blue','brown','green','grey','orange','pink','purple','red','white','yellow']

# For each color, compute the cosine similarity and store in a new column
for color in color_list:
    bless[color] = bless['noun'].apply(lambda noun: compute_cosine_similarity(noun, color))

bless['Max_similarity'] = bless[color_list].max(axis=1)
bless['Max_color'] = bless[color_list].idxmax(axis=1)
bless['accuracy'] = (bless['color'] == bless['Max_color']).astype(int)
bless

,noun,color,true_similarity,black,blue,brown,green,grey,orange,pink,purple,red,white,yellow,Max_similarity,Max_color,accuracy
0,ash,grey,0.948917,0.935817,0.933177,0.928751,0.935453,0.948917,0.939263,0.914402,0.937057,0.915655,0.936745,0.930945,0.948917,grey,1
1,banana,yellow,0.906686,0.900736,0.911862,0.900141,0.898841,0.906241,0.933359,0.876197,0.908878,0.882756,0.906549,0.906686,0.933359,orange,0
2,beaver,brown,0.918501,0.925109,0.923486,0.918501,0.923626,0.932245,0.927683,0.906457,0.939775,0.904467,0.930039,0.913888,0.939775,purple,0
3,blood,red,0.811559,0.841519,0.823208,0.817501,0.821375,0.846251,0.820037,0.844922,0.846017,0.811559,0.822205,0.804603,0.846251,grey,0
4,broccoli,green,0.799139,0.788965,0.791254,0.810630,0.799139,0.796576,0.814399,0.796235,0.801844,0.772710,0.800255,0.792570,0.814399,orange,0
5,cardboard,brown,0.853881,0.862008,0.858008,0.853881,0.862500,0.864234,0.877746,0.845690,0.866892,0.852489,0.869051,0.868634,0.877746,orange,0
6,carrot,orange,0.885415,0.867952,0.873695,0.848660,0.867019,0.867478,0.885415,0.853352,0.881557,0.861724,0.873314,0.872614,0.885415,orange,1
7,cauliflower,white,0.700411,0.684517,0.698634,0.699868,0.686658,0.698200,0.714194,0.673171,0.713525,0.662025,0.700411,0.695553,0.714194,orange,0
8,cello,brown,0.883016,0.895316,0.903640,0.883016,0.890628,0.899351,0.900418,0.898764,0.905642,0.880784,0.897548,0.905023,0.905642,purple,0
9,chalk,white,0.912911,0.910691,0.884853,0.882013,0.904037,0.897737,0.880037,0.852338,0.887975,0.880149,0.912911,0.883134,0.912911,white,1


In [9]:
overall_accuracy = bless['accuracy'].sum() / len(bless)

print(f"Overall Accuracy: {overall_accuracy:.2%}")

Overall Accuracy: 13.46%


In [10]:
# For each row, rank the cosine similarities and determine the rank of the true color
ranks = []
for index, row in bless.iterrows():
    ranked_colors = row[color_list].sort_values(ascending=False).index.tolist()
    rank_of_true_color = ranked_colors.index(row['color']) + 1
    ranks.append(rank_of_true_color)

# Add ranks to the dataframe
bless['rank'] = ranks

# Calculate median rank
median_rank = pd.Series(ranks).median()

print("Median Rank:", median_rank)

Median Rank: 5.0


In [11]:
!pip install deep_translator
from deep_translator import GoogleTranslator
to_translate = 'I want to translate this text'
def translate_text(text):
  return GoogleTranslator(source='auto', target='es').translate(text)
# outpout -> Ich möchte diesen Text übersetzen
#from googletrans import Translator

def translate_column_to_spanish(df, column_name1, column_name2):
    # Initialize the translator


    # Translate the words in the specified column to Spanish
    df['nombre_esp'] = df[column_name1].apply(lambda text: translate_text(text,))
    df['color_esp'] = df[column_name2].apply(lambda text: translate_text(text,))

    return df


# Call the translate_column_to_spanish function
translated_df = translate_column_to_spanish(df, 'noun', 'color')


# Display the translated DataFrame
print(translated_df)
bless = translated_df




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00
           noun   color  true_similarity     black      blue     brown  \
0           ash    grey         0.948917  0.935817  0.933177  0.928751   
1        banana  yellow         0.906686  0.900736  0.911862  0.900141   
2        beaver   brown         0.918501  0.925109  0.923486  0.918501   
3         blood     red         0.811559  0.841519  0.823208  0.817501   
4      broccoli   green         0.799139  0.788965  0.791254  0.810630   
5     cardboard   brown         0.853881  0.862008  0.858008  0.853881   
6        carrot  orange         0.885415  0.867952  0.873695  0.848660   
7   cauliflower   white         0.700411  0.684517  0.698634  0.699868   
8         cello   brown         0.883016  0.895316  0.903640  0.883016   
9         chalk   white         0.912911  0.910691  0.884853  0.882013   
10       cherry     red         0.904697  0.946930  0.934454  0.924183   
11        cloud   white         

In [12]:
print(bless['nombre_esp'].isnull().sum())
print(bless['color_esp'].isnull().sum())

0
0


In [13]:
embedding1 = get_bert_embeddings(text1)
embedding2 = get_bert_embeddings(text2)

def compute_cosine_similarity(noun, color):
    noun_vector = get_bert_embeddings(noun)
    color_vector = get_bert_embeddings(color)
    noun_vector_np = noun_vector.numpy()
    color_vector_np = color_vector.numpy()
    similarity = cosine_similarity(noun_vector_np, color_vector_np)
    return similarity[0][0]

# Add true_similarity column
bless['true_similarity'] = bless.apply(lambda row: compute_cosine_similarity(row['nombre_esp'], row['color_esp']), axis=1)

color_list = ['gris', 'amarillo', 'marrón', 'rojo', 'verde', 'naranja', 'blanco', 'negro', 'rosa', 'azul']

# For each color, compute the cosine similarity and store in a new column
for color in color_list:
    bless[color] = bless['nombre_esp'].apply(lambda noun: compute_cosine_similarity(noun, color))

bless['Max_similarity'] = bless[color_list].max(axis=1)
bless['Max_color'] = bless[color_list].idxmax(axis=1)
bless['accuracy'] = (bless['color_esp'] == bless['Max_color']).astype(int)
bless

,noun,color,true_similarity,black,blue,brown,green,grey,orange,pink,...,gris,amarillo,marrón,rojo,verde,naranja,blanco,negro,rosa,azul
0,ash,grey,0.876429,0.935817,0.933177,0.928751,0.935453,0.948917,0.939263,0.914402,...,0.876429,0.869012,0.884035,0.872679,0.887728,0.843676,0.905214,0.893179,0.911007,0.906859
1,banana,yellow,0.848018,0.900736,0.911862,0.900141,0.898841,0.906241,0.933359,0.876197,...,0.908182,0.848018,0.924027,0.870185,0.919285,0.871129,0.901391,0.910977,0.916791,0.902913
2,beaver,brown,0.789108,0.925109,0.923486,0.918501,0.923626,0.932245,0.927683,0.906457,...,0.769669,0.743354,0.789108,0.769050,0.784224,0.721807,0.780678,0.781331,0.811364,0.776007
3,blood,red,0.871494,0.841519,0.823208,0.817501,0.821375,0.846251,0.820037,0.844922,...,0.778751,0.900499,0.805690,0.871494,0.813912,0.746159,0.877609,0.861313,0.848553,0.880647
4,broccoli,green,0.891585,0.788965,0.791254,0.810630,0.799139,0.796576,0.814399,0.796235,...,0.890917,0.833924,0.879898,0.843178,0.891585,0.848752,0.887046,0.878083,0.893705,0.868293
5,cardboard,brown,0.831498,0.862008,0.858008,0.853881,0.862500,0.864234,0.877746,0.845690,...,0.823545,0.782909,0.831498,0.801035,0.832212,0.812519,0.833703,0.830979,0.861319,0.833300
6,carrot,orange,0.791357,0.867952,0.873695,0.848660,0.867019,0.867478,0.885415,0.853352,...,0.796124,0.817433,0.820263,0.810689,0.829196,0.791357,0.834901,0.820850,0.838790,0.831338
7,cauliflower,white,0.783397,0.684517,0.698634,0.699868,0.686658,0.698200,0.714194,0.673171,...,0.776439,0.755847,0.773332,0.772519,0.785345,0.762493,0.783397,0.769545,0.802703,0.778616
8,cello,brown,0.556705,0.895316,0.903640,0.883016,0.890628,0.899351,0.900418,0.898764,...,0.543370,0.564318,0.556705,0.580774,0.571810,0.545904,0.568958,0.576184,0.574839,0.578667
9,chalk,white,0.889725,0.910691,0.884853,0.882013,0.904037,0.897737,0.880037,0.852338,...,0.908651,0.818621,0.895992,0.838721,0.904957,0.870413,0.889725,0.893495,0.897456,0.881876


In [14]:
overall_accuracy = bless['accuracy'].sum() / len(bless)

print(f"Overall Accuracy: {overall_accuracy:.2%}")

Overall Accuracy: 11.54%


In [15]:
# For each row, rank the cosine similarities and determine the rank of the true color
ranks = []
for index, row in bless.iterrows():
    ranked_colors = row[color_list].sort_values(ascending=False).index.tolist()
    rank_of_true_color = ranked_colors.index(row['color_esp']) + 1
    ranks.append(rank_of_true_color)

# Add ranks to the dataframe
bless['rank'] = ranks

# Calculate median rank
median_rank = pd.Series(ranks).median()

print("Median Rank:", median_rank)

Median Rank: 5.0
